# Import libraries
We are going to use a deep-learning framework known as PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

Use GPU if available.

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

# Dataset and Loader

In [ ]:
batch_size = 128

# Create a data loader to feed data for training.
# transform performs operations on data before feeding it to network. Operations
# like Normalization, Crop, Flip, Resize and many more can be performed.
# Ref: https://pytorch.org/vision/0.9/transforms.html
# 0.1307 is mean and 0.3081 is std of our data set
# use shuffle = True to shuffle data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Create a data loader to feed data for testing.
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [ ]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in:3 , n_in:28 , j_in:1 , s:1 , r_out:5 , n_out:28 , j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:28 , j_in:1 , s:2 , r_out:6 , n_out:14 , j_out:2
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in:6 , n_in:14 , j_in:2 , s:1 , r_out:10 , n_out:14 , j_out:2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in:10 , n_in:14 , j_in:2 , s:1 , r_out:14 , n_out:14 , j_out:2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in:14 , n_in:14 , j_in:2 , s:2 , r_out:16 , n_out:7 , j_out:4
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in:16 , n_in:7 , j_in:4 , s:1 , r_out:24 , n_out:5 , j_out:4
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in:24 , n_in:5 , j_in:4 , s:1 , r_out:32 , n_out:3 , j_out:4
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in:32 , n_in:3 , j_in:4 , s:2 , r_out:40 , n_out:1 , j_out:4
    self.conv7 = nn.Conv2d(1024, 10, 3)
# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    #x = F.relu(x) # this is the last step. Think what ReLU does to our results at this stage!
    x = x.view(-1, 10)
    return F.log_softmax(x)


# Initialize model

In [ ]:
model = FirstDNN().to(device)

In [ ]:
# Print model summary. Look at trainable/non-trainable parameters and model size.
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-4-0c2f7993ea4d>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


# Training and validation methods

In [ ]:
# Import tqdm to print progress bar while iterating through data
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    """
    Method to train model.
    Get prediction from model. -> Calculate loss. -> Update weights
    Args:
    ---------
    model: instance of network
    device: device to use for calculation (GPU/CPU)
    train_loader: traning data loader
    optimizer: optimizer to update weights
    epoch: Current epoch
    """
    # Set model to train
    model.train()
    pbar = tqdm(train_loader)

    # iterate though data and update model weight as required
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) # get prediction
        loss = F.nll_loss(output, target) # calculate loss
        loss.backward() # compute gradients
        optimizer.step() # update weights
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    """
    Method to test model.
    Get prediction from model. -> Calculate loss and accuracy.
    Args:
    ---------
    model: instance of network
    device: device to use for calculation (GPU/CPU)
    test_loader: testing data loader
    """
    # Set model to train
    model.eval()
    test_loss = 0
    correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # iterate though data and calculate loss
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data) # get prediction
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# Create optimizer object.
# lr is learning rate.
# momentum helps in speeding up traning speed and reduce possibility of training getting stuck around local minima
# ref: https://medium.com/@vinodhb95/momentum-optimizer-6023aa445e18, https://www.kdnuggets.com/2022/12/tuning-adam-optimizer-parameters-pytorch.html
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Run training for 1 epoch
for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-4-0c2f7993ea4d>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06928126513957977 batch_id=468: 100%|██████████| 469/469 [00:31<00:00, 14.80it/s]



Test set: Average loss: 0.0559, Accuracy: 9822/10000 (98%)

